# Let's get interactive!

## Interactivity

In addition to basic plotting and view composition, one of Altair and Vega-Lite's most exciting features is its support for interaction.

To create a simple interactive plot that supports panning and zooming, we can invoke the `interactive()` method of the `Chart` object. In the chart below, click and drag to *pan* or use the scroll wheel to *zoom*:

In [27]:
import pandas as pd
import altair as alt

In [28]:
from vega_datasets import data  # import vega_datasets
cars = data.cars()              # load cars data as a Pandas data frame
cars.head()                     # display the first five rows

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


In [29]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
).interactive()

alt.Chart(...)

To provide more details upon mouse hover, we can use the `tooltip` encoding channel:

In [31]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin', 'Displacement'] # show Name and Origin in a tooltip
).interactive()

alt.Chart(...)

In [39]:
slider = alt.binding_range(min=0, max=1, step=0.05, name='opacity:')
op_var = alt.param(value=0.1, bind=slider)

alt.Chart(cars).mark_circle(opacity=op_var).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    size = op_var
).add_params(
    op_var
)

alt.Chart(...)

For more complex interactions, such as linked charts and cross-filtering, Altair provides a *selection* abstraction for defining interactive selections and then binding them to components of a chart. We will cover this is in detail in a later notebook.

Below is a more complex example. The upper histogram shows the count of cars per year and  uses an interactive selection to modify the opacity of points in the lower scatter plot, which shows horsepower versus mileage.

_Drag out an interval in the upper chart and see how it affects the points in the lower chart. 

In [50]:
# create an interval selection over an x-axis encoding
brush = alt.selection_interval(encodings=['x'], resolve = 'union')

# determine opacity based on brush
opacity = alt.condition(brush, alt.value(0.9), alt.value(0.1))

# an overview histogram of cars per year
# add the interval brush to select cars over time
overview = alt.Chart(cars).mark_bar().encode(
    alt.X('Year:O', timeUnit='year', # extract year unit, treat as ordinal
      axis=alt.Axis(title=None, labelAngle=0) # no title, no label angle
    ),
    alt.Y('count()', title=None), # counts, no axis title
    opacity=opacity
).add_params(
    brush      # add interval brush selection to the chart
).properties(
    width=400, # set the chart width to 400 pixels
    height=50  # set the chart height to 50 pixels
)

# a detail scatterplot of horsepower vs. mileage
# modulate point opacity based on the brush selection
detail = alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    # set opacity based on brush selection
    opacity=opacity
).properties(width=400) # set chart width to match the first chart

# vertically concatenate (vconcat) charts using the '&' operator
overview & detail

alt.VConcatChart(...)

The following is a different behaviour associated with the interaction. Remember the **crossfilter** operation from the slides? A selection in one view filters the other. The selections in the scatterplot determines what the histogram is displaying. Go ahead and make some selections in the scatterplot.

In [51]:
source = data.cars()

brush = alt.selection_interval()

points = alt.Chart(source).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
).add_params(
    brush
)

bars = alt.Chart(source).mark_bar().encode(
    y='Origin:N',
    color='Origin:N',
    x='count(Origin):Q'
).transform_filter(
    brush
)

points & bars

alt.VConcatChart(...)

**Question:** What does the interaction achieve here? Does it make new ways of analysing the data possible?

The following is similar to the above ones but this time uses two linked views. Experiment here with the "resolve" parameter.  See the documentation https://altair-viz.github.io/user_guide/generated/api/altair.selection_interval.html for alternatives, what do they do?

In [54]:
import altair as alt
from vega_datasets import data

source = data.cars()

brush = alt.selection_interval(resolve='intersect')

base = alt.Chart(source).mark_point().encode(
    y='Miles_per_Gallon',
    color=alt.condition(brush, 'Origin', alt.ColorValue('gray')),
).add_params(
    brush
).properties(
    width=250,
    height=250
)

base.encode(x='Horsepower') | base.encode(x='Acceleration', y='Weight_in_lbs')

alt.HConcatChart(...)

In [76]:
brush = alt.selection_interval(resolve='global')
opac = alt.condition(brush, alt.value(0.9), alt.value(0.1))
base = alt.Chart(source).mark_point().encode(
    y='Miles_per_Gallon',
    color=alt.condition(brush, 'Origin', alt.ColorValue('gray')),
    opacity=opac
).add_params(
    brush
).properties(
    width=250,
    height=250
).transform_calculate(rand="random()")

base.encode(x='Year', y='Origin', yOffset="rand:Q") | base.encode(x='Acceleration')

alt.HConcatChart(...)

In [74]:
myInteractiveChart

alt.HConcatChart(...)

In [57]:
myInteractiveChart.save('chart.html')

In [61]:
brush = alt.selection_interval(encodings=['x'])
base = alt.Chart(source).mark_point().encode(
    y='Miles_per_Gallon',
    color=alt.condition(brush, 'Origin', alt.ColorValue('gray')),
).add_params(
    brush
).properties(
    width=250,
    height=250
)

base.mark_bar().encode(x='Cylinders', y='mean_acc:Q').transform_aggregate(
    mean_acc='mean(Acceleration)',
    groupby=["Cylinders"]
) | base.encode(x='Horsepower')


alt.HConcatChart(...)

<img src="tools-hammer.svg" width="60">  **GIVE IT A TRY!** 

Can you transfer the scatterplot matrix you generated in the previous exercise and make it a linked and interactive scatterplot matrix?

In [ ]:
from vega_datasets import data as vega_data

incomeHealthData = vega_data.gapminder_health_income()
incomeHealthData.head()

In [67]:
brush = alt.selection_interval(resolve='intersect')

scatters = alt.Chart(incomeHealthData).mark_point().encode(
alt.X(alt.repeat("column"), type='quantitative'),
alt.Y(alt.repeat("row"), type='quantitative'),
color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'), legend=None),
tooltip = ['country']
).add_params(
    brush
).properties(
    width=200,
    height=200
).repeat(
row=['income','health','population'],
column=['income', 'health','population'],
)

scatters

alt.RepeatChart(...)

<img src="tools-hammer.svg" width="60">  **GIVE IT A TRY!** 

If you managed to go through the above quickly and have a bit of time until the break, how about making a mini interactive dashboard? We can use another Vega dataset, the weather data.

In [68]:
weather = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

Start first by visualising the data to get to know it better. And then start building a few blocks that can be part of a larger "dashboard. Use view combination approaches to put together a design for a composite visualisation.

In [69]:
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q')
)

alt.Chart(...)

In [70]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N')
).properties(
  width=150,
  height=150
)

alt.Chart(...)